## Import Libraries

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from random import randint
from time import sleep
from itertools import islice
from pandas import json_normalize
import numpy as np
import pickle

## Spotify Authentication

In [2]:
# Read secrets from a file to use for Spotify authentication
secrets_file = open("secrets.txt","r")

In [3]:
string = secrets_file.read()
#string

In [19]:
#string.split('\n')

In [5]:
# Split the secrets string into a dictionary for easier access
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        #print(line.split(':'))
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()
        
#secrets_dict

In [6]:
#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))

## Loading Machine Learning Models

In [7]:
# Load the KMeans model from a pickle file
with open('kmeans.pkl', 'rb') as file:
    kmeans = pickle.load(file)

# Load the scaler
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

### Verifying the KMeans Model

In [8]:
# Print the shape of the KMeans cluster centers to verify the model is loaded correctly
print(kmeans.cluster_centers_.shape)

(17, 11)


## Loading DataFrames

In [9]:
# # Load the final DataFrame from a CSV file (the one with the clusters)
df_final = pd.read_csv('df_final.csv')

# # Load the top songs from a CSV file (the one with the hot songs)
top_songs = pd.read_csv('topsongs_final.csv')

In [10]:
df_final.head()

,artists,song,cluster
0,Gen Hoshino,Comedy,5
1,Ben Woodward,Ghost - Acoustic,16
2,Ingrid Michaelson;ZAYN,To Begin Again,16
3,Kina Grannis,Can't Help Falling In Love,3
4,Chord Overstreet,Hold On,16


In [11]:
top_songs.head()

,song,artists
0,Cruel Summer,Taylor Swift
1,Paint The Town Red,Doja Cat
2,Is It Over Now? (Taylor's Version) [From The V...,Taylor Swift
3,Snooze,SZA
4,Standing Next To You,Jung Kook


## Song Recommender

### Explanation:

#### First of all: The function song_recommender takes a song name as input and then:

1. It first checks if the song is in the top_songs DataFrame's Hot 100 list (case-insensitively).
2. If the song is in the Hot 100, it recommends another song from that list.
3. If the song is not in the Hot 100, the function searches for it on Spotify.
4. The user is asked to confirm if the first search result (song and artist) is correct. If not, the search continues with an offset to get the next result.
5. Once the correct song is found, its audio features are fetched from Spotify.
6. These features are scaled (with the pre-fitted scaler) and used to predict the cluster of the song using a pre-trained KMeans model (assumed as kmeans).
7. Finally, the function recommends a random song from the same cluster in the df_final DataFrame.

In [15]:
def song_recommender(song_name):
    """
    Recommends a song based on the input song name.

    This function first checks if the input song is in the PopVortex Hot 100 list. If it is,
    another song from the Hot 100 list is recommended. If not, the function searches for the song
    on Spotify, confirms the artist with the user, and then recommends a song from the same cluster
    based on audio features obtained from Spotify.

    Args:
    song_name (str): The name of the song input by the user.

    Returns:
    None: The function prints the recommendation but does not return any value.
    """
    # Check if the song is in the PopVortex Hot 100 (case-insensitive)
    if top_songs['song'].str.lower().eq(song_name.lower()).any():
        recommendation = top_songs.sample()
        print(f"The song is in the Hot 100! Try this one instead: '{recommendation['song'].iloc[0]}' by {recommendation['artists'].iloc[0]}")
    else:
        search_done = False
        search_offset = 0

        while not search_done:
            # Search for the song on Spotify
            results = sp.search(q=song_name, type='track', limit=1, offset=search_offset)
            if not results['tracks']['items']:
                print("The song was not found on Spotify.")
                return
            
            track = results['tracks']['items'][0]
            track_id = track['id']
            track_name = track['name']
            track_artist = track['artists'][0]['name']

            # Ask the user for confirmation about the artist
            user_confirmation = input(f"Is this your song: '{track_name}' by {track_artist}? (yes/no): ").lower()
            if user_confirmation == 'yes':
                search_done = True
            else:
                search_offset += 1

        audio_features = sp.audio_features(track_id)[0]
        features = np.array([[audio_features[feature] for feature in [
            'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
            'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo'
        ]]])

        # Scale the features (assuming scaler is previously fitted)
        scaled_features = scaler.transform(features)

        # Predict the cluster for the song
        cluster = kmeans.predict(scaled_features)[0]

        # Recommend a song from the same cluster
        same_cluster_songs = df_final[df_final['cluster'] == cluster]
        if same_cluster_songs.empty:
            print("No recommendation available from the same cluster.")
            return
        
        recommended_song = same_cluster_songs.sample()
        print(f"Try this song: '{recommended_song['song'].iloc[0]}' by {recommended_song['artists'].iloc[0]} from cluster {cluster}.")

In [17]:
# Example use of the function
song_name_input = input("Enter a song you like: ")
song_recommender(song_name_input)

Enter a song you like: Footsteps
Is this your song: 'Footsteps in the Dark, Pts. 1 & 2' by The Isley Brothers? (yes/no): no
Is this your song: 'Footsteps' by Pop Evil? (yes/no): no
Is this your song: 'Footsteps (feat. Musiq Soulchild)' by Kehlani? (yes/no): yes
Try this song: 'Real Thing' by Pink Sweat$;Tori Kelly from cluster 16.


/Users/mac/anaconda3/envs/nahuelito/lib/python3.9/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/mac/anaconda3/envs/nahuelito/lib/python3.9/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(


In [18]:
# Example use of the function
song_name_input = input("Enter a song you like: ")
song_recommender(song_name_input)

Enter a song you like: Snooze
The song is in the Hot 100! Try this one instead: '3D' by Jung Kook & Jack Harlow
